In [13]:
# import pandas module
import pandas as pd
import numpy as np
from pandas.core.accessor import CachedAccessor

In [14]:
# create pandas dataframe
CCData = pd.read_csv('application_data.csv')

In [6]:
# view CCData dataframe info
print(CCData.info(verbose=True, show_counts=True))
# create clean Data Frame (drop rows with null values)
CCData_clean = CCData.dropna(axis='rows')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 122 columns):
 #    Column                        Non-Null Count   Dtype  
---   ------                        --------------   -----  
 0    SK_ID_CURR                    307511 non-null  int64  
 1    TARGET                        307511 non-null  int64  
 2    NAME_CONTRACT_TYPE            307511 non-null  object 
 3    CODE_GENDER                   307511 non-null  object 
 4    FLAG_OWN_CAR                  307511 non-null  object 
 5    FLAG_OWN_REALTY               307511 non-null  object 
 6    CNT_CHILDREN                  307511 non-null  int64  
 7    AMT_INCOME_TOTAL              307511 non-null  float64
 8    AMT_CREDIT                    307511 non-null  float64
 9    AMT_ANNUITY                   307499 non-null  float64
 10   AMT_GOODS_PRICE               307233 non-null  float64
 11   NAME_TYPE_SUITE               306219 non-null  object 
 12   NAME_INCOME_TYPE            

In [16]:
# create dummy variablies form CCData_clean
dummy = pd.get_dummies(CCData_clean.select_dtypes('object'))
# combine the dummy variables with CCData_clean
CCData_concat = pd.concat([CCData_clean, dummy], axis=1)
# drop object datatypes for numerical Data Frame
CCData_numerical = CCData_concat.select_dtypes(exclude='object')
print(CCData_numerical)


        SK_ID_CURR  TARGET  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
71          100083       0             0          103500.0    573628.5   
124         100145       0             1          202500.0    260725.5   
152         100179       0             0          202500.0    675000.0   
161         100190       0             0          162000.0    263686.5   
255         100295       1             1          225000.0   1019205.0   
...            ...     ...           ...               ...         ...   
307358      456083       0             2          112500.0    361462.5   
307359      456084       0             1           99000.0    675000.0   
307407      456140       1             1          261000.0    711454.5   
307456      456195       0             0           94500.0    270000.0   
307482      456226       0             0          225000.0    500566.5   

        AMT_ANNUITY  AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  DAYS_BIRTH  \
71          24435.0         463

In [17]:
# import seaborn and matplotlib
import seaborn as sn
import matplotlib.pyplot as plt 


# Create a correlation matrix to detect highly correlelated predictor variables
corrMatrix= CCData_numerical.corr().abs()

# select upper traingle of correlation matrix

upper = corrMatrix.where(np.triu(np.ones(corrMatrix.shape), k=1).astype(np.bool))

# identify correlatons greater than abs(.8)

to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]

CCData_final = CCData_numerical.drop(CCData_numerical[to_drop], axis=1)
print(CCData_final)
# see final percentage of target variable before SMOTE = .061149 or 6.1149% (lower % than original data)
print(CCData_final.TARGET.value_counts()/8602)

# save CCData_file as a excel file
CCData_final.to_excel('out.xlsx', sheet_name='CCData_final')


# create a heatmap of the correlation matrix
#sn.heatmap(corrMatrix, annot=True)
#plt.show()

        SK_ID_CURR  TARGET  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
71          100083       0             0          103500.0    573628.5   
124         100145       0             1          202500.0    260725.5   
152         100179       0             0          202500.0    675000.0   
161         100190       0             0          162000.0    263686.5   
255         100295       1             1          225000.0   1019205.0   
...            ...     ...           ...               ...         ...   
307358      456083       0             2          112500.0    361462.5   
307359      456084       0             1           99000.0    675000.0   
307407      456140       1             1          261000.0    711454.5   
307456      456195       0             0           94500.0    270000.0   
307482      456226       0             0          225000.0    500566.5   

        AMT_ANNUITY  REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED  \
71          24435.0              

import statsmodels.api as sm
"""

x = CCData_final[CCData_final.columns[CCData_final.columns != 'TARGET']]
y = CCData_final.TARGET
 
# Statsmodels.OLS requires us to add a constant.
x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit()
print(results.summary())

"""

def prep_data (CCData_final: pd.DataFrame) -> (np.array,np.ndarray):
    """
    convert the DataFrame into two variables
    """
    X = CCData_final.iloc[:, 2:235].values
    y = CCData_final.TARGET.values
    return X, y 


import matplotlib.pyplot as plt
# Define a function to create a scatter plot of our data and labels
def plot_data(X: np.ndarray, y: np.ndarray):
	plt.scatter(X[y == 0, 0], X[y == 0, 1], label="TARGET #0", alpha=0.5, linewidth=0.15)
	plt.scatter(X[y == 1, 0], X[y == 1, 1], label="TARGET #1", alpha=0.5, linewidth=0.15, c='r')
	plt.legend()
	return plt.show()

# Create X and y from the prep_data function 
X, y = prep_data(CCData_final)

# Plot our data by running our plot data function on X and y
plot_data(X, y)